In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

AUTOTUNE = tf.data.experimental.AUTOTUNE
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd 'drive/My Drive/dataset/cassava_leaf_disease'

/content/drive/My Drive/dataset/cassava_leaf_disease


In [ ]:
ls

test/  train/


In [5]:
#get data
train_paths=[]
train_labels=[]
for root,dir,files in os.walk("train"):
    for file in files:
        train_paths.append(os.path.join(root,file))
        train_labels.append(root.split("/")[-1])

test_paths=[]
for root,dir,files in os.walk("test"):
    for file in files:
        test_paths.append(os.path.join(root,file))

In [6]:
#function to preprocess images and labels
def preprocess_image(image,image_shape=[192,192]):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_shape)
    image /= 255.0  # normalize to [0,1] range
    
    return image


def load_and_preprocess_image(file,image_shape):
    image = tf.io.read_file(file)
    return preprocess_image(image,image_shape)

labs2index={'cbb':0, 'cbsd':1, 'cgm':2, 'cmd':3, 'healthy':4}
index2labs={0:'cbb', 1:'cbsd', 2:'cgm', 3:'cmd', 4:'healthy'}


def process_labels(lab):
    print(lab)
    return tf.one_hot(lab,depth=len(labs2index))

In [7]:
#create tensorflow data objects
train_ds = tf.data.Dataset.from_tensor_slices(train_paths)
test_ds=tf.data.Dataset.from_tensor_slices(test_paths)

train_image_ds = train_ds.map(lambda x:load_and_preprocess_image(x,image_shape=[192,192]),num_parallel_calls=AUTOTUNE)
test_image_ds=test_ds.map(lambda x:load_and_preprocess_image(x,image_shape=[192,192]),num_parallel_calls=AUTOTUNE)

train_label_ds=tf.data.Dataset.from_tensor_slices([labs2index[i] 
                                                   for i in train_labels]).map(process_labels,num_parallel_calls=AUTOTUNE)
image_label_ds=tf.data.Dataset.zip((train_image_ds,train_label_ds))

BATCH_SIZE = 32

# Setting a shuffle buffer size as large as the dataset ensures that the data is
# completely shuffled.
ds = image_label_ds.shuffle(buffer_size=len(train_paths))
ds = ds.repeat()
ds = ds.batch(BATCH_SIZE)
# `prefetch` lets the dataset fetch batches, in the background while the model is training.
ds = ds.prefetch(buffer_size=AUTOTUNE)

ds_test=test_image_ds.batch(BATCH_SIZE)
print(ds)
print(test_ds)

Tensor("args_0:0", shape=(), dtype=int32)
<PrefetchDataset shapes: ((None, 192, 192, 3), (None, 5)), types: (tf.float32, tf.float32)>
<TensorSliceDataset shapes: (), types: tf.string>


In [8]:
#use pretrained model
vgg19 = tf.keras.applications.VGG19(input_shape=(192, 192, 3), include_top=False)

def change_range(image,label):
    return 2*image-1, label

80142336/80134624 [==============================] - 1s 0us/step


In [9]:
keras_ds = ds.map(change_range)
keras_test=ds_test.map(lambda x:2*x-1)

#image_batch, label_batch = next(iter(keras_ds))

model = tf.keras.Sequential([vgg19, tf.keras.layers.GlobalAveragePooling2D(),
                             tf.keras.layers.Dense(len(labs2index),activation=tf.nn.softmax)])

model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=["accuracy"])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 6, 6, 512)         20024384  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 5)                 2565      
Total params: 20,026,949
Trainable params: 20,026,949
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
model.fit(keras_ds, epochs=4, steps_per_epoch=10)

Epoch 1/4
10/10 [==============================] - 733s 51s/step - loss: 1.6691 - accuracy: 0.2662
Epoch 2/4
10/10 [==============================] - 511s 51s/step - loss: 1.5957 - accuracy: 0.2833
Epoch 3/4
10/10 [==============================] - 505s 51s/step - loss: 1.4712 - accuracy: 0.3555
Epoch 4/4
10/10 [==============================] - 504s 50s/step - loss: 1.5131 - accuracy: 0.3454


In [11]:
cd '/content/drive/My Drive/ML_models_tflite'

/content/drive/My Drive/ML_models_tflite


In [12]:
from tensorflow import lite

converter = lite.TFLiteConverter.from_keras_model(model)
tfmodel = converter.convert()
open('cassava_plant_tf.tflite', 'wb').write(tfmodel)

INFO:tensorflow:Assets written to: /tmp/tmpddpc3mrx/assets


80119316